In [4]:
import tensorflow as tf
import numpy
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Define our input values

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

## Deffining our initial weights

In [6]:
def weight_variable(shape):
    w_h = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(w_h)
def bias_variable(shape):
    b_h = tf.constant(0.1, shape = shape)
    return tf.Variable(b_h)

## Deffining useful functions

In [7]:
def conv2d(Input, Weights):
    return tf.nn.conv2d(Input, Weights, strides = [1, 1, 1, 1],
                        padding='SAME')
def max_pool_2x2(Input):
    return tf.nn.max_pool(Input, ksize = [1, 2, 2, 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

In [8]:
def cnn_layer(Input, Weights_shape, Bias_shape):
    w_conv = weight_variable(Weights_shape)
    b_conv = weight_variable(Bias_shape)
    h_conv = tf.nn.relu(conv2d(Input, w_conv) + b_conv)
    return h_conv, w_conv, b_conv

## CapsT

In [9]:
#Inputs = conv output
#n_convcaps = 8
#n_channels = 32
#kernel_size = 9
#n_strides = 2
#padding = 'valid' 


In [10]:
def conv2d(Input, Weights, n_strides=2, padding='SAME'):
    return tf.nn.conv2d(Input, Weights, strides = [1, n_strides, n_strides, 1], padding=padding)
def max_pool_2x2(Input):
    return tf.nn.max_pool(Input, ksize = [1, 2, 2, 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [12]:
def cnn_layer(Input, Weights_shape, Bias_shape, n_strides, padding, activation=True):
    w_conv = weight_variable(Weights_shape)
    b_conv = weight_variable(Bias_shape)
    if activation == True:
        h_conv = tf.nn.relu(conv2d(Input, w_conv, n_strides, padding) + b_conv)
    else:
        h_conv = conv2d(Input, w_conv, n_strides, padding) + b_conv
    return h_conv, w_conv, b_conv

In [13]:
def Primary_caps(Inputs, n_convcaps, n_channels, kernel_size, n_strides, padding):
    h_conv, W_conv, b_conv = cnn_layer(Inputs, kernel_size, [n_convcaps*n_channels], n_strides, padding, activation='False')
    caps_outs = tf.reshape(h_conv, [(Inputs.shape)[0], -1, n_convcaps])
    return caps_outs, h_conv

In [14]:
def squashing_func(vector, axis=-1):
    sq_norm = tf.reduce_sum(tf.square(vector),0, keep_dims=True);
    first_eq = sq_norm/(1+sq_norm);
    second_eq = vector / tf.sqrt(sq_norm);
    return first_eq*second_eq

In [21]:
def digcaps(Input, o_size, v_len):
    print Input.shape
    r_Input = tf.reshape(Input, [(Input.shape)[0], -1, 1, (Input.shape)[2]])
    b_I = tf.constant(np.zeros([1, Input.shape[1].value, o_size, 1], dtype=np.float32))
    print r_Input.shape, b_I.shape
    W = weight_variable([1, 1152, 10, 8, 16])
    print W.shape
    t_Input = tf.tile(Input,[1, 1, 10])
    W= tf.tile(W, [Input.shape[0], 1, 1, 1, 1])
    print t_Input.shape
    print W.shape
    return r_Input

In [22]:

cfg = tf.app.flags.FLAGS
tf.Session.reset
with tf.device('/gpu:1'): #or '/cpu:0' for cpu only
    x_image = tf.reshape(x, [-1, 28, 28, 1])    
    h_conv1, W_conv1, b_conv1 = cnn_layer(x_image, [9, 9, 1, 256], [256], 1, padding='VALID')
    #h_conv, W_conv, b_conv = cnn_layer(h_conv1, [9, 9, 256, 32*8], [32*8], 2, padding='VALID', activation='False')
    primarycaps, h_pout = Primary_caps(h_conv1, 8, 32, [9, 9, 256, 32*8], 2,padding='VALID')
    #primarycaps_squashed = squashing_func(primarycaps);
    #digitCaps = digcaps(primarycaps_squashed, 10, 16);
    #h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    b_ij = tf.constant(0, shape = [1, 10, 10, 1,1])
    #c = tf.nn.softmax(b_ij, dim =2)
    
    with tf.Session(config = tf.ConfigProto(allow_soft_placement = True,
                                           log_device_placement = True)) as sess:
        
        x = mnist.train.images[0:2]
        xx = x_image
        hh = h_conv1
        h_pouts = h_pout
        prim = primarycaps
        sqhd = primarycaps_squashed
        bj = b_ij
        

TypeError: Failed to convert object of type <type 'list'> to Tensor. Contents: [Dimension(None), -1, 8]. Consider casting elements to a supported type.

In [17]:
print x.shape
print xx.shape
print h_pouts.shape
print h_conv1.shape
print prim.shape
print bj.shape
print sqhd.shape

(?, 784)


NameError: name 'xx' is not defined

In [18]:
tf.shape

<function tensorflow.python.ops.array_ops.shape>